<a href="https://colab.research.google.com/github/nothing2c/AI-Assignment-3/blob/main/AI_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>